In [1]:
# Path setup so Python can find the compiled _earcore module and the Python package
import sys
from pathlib import Path

def add_paths():
    cwd = Path.cwd()
    # Walk up to locate eartrainer/eartrainer_Cpp/{build,python}
    for base in [cwd, *cwd.parents]:
        build = base / 'eartrainer' / 'eartrainer_Cpp' / 'build'
        py = base / 'eartrainer' / 'eartrainer_Cpp' / 'python'
        added = False
        if build.exists():
            sys.path.insert(0, str(build))
            added = True
        if py.exists():
            sys.path.insert(0, str(py))
            added = True
        if added:
            return build, py
    return None, None

build_path, py_path = add_paths()
print('Using build path:', build_path)
print('Using python path:', py_path)


Using build path: /Users/itamarshamir/Projects/ear_trainer/eartrainer/eartrainer_Cpp/build
Using python path: /Users/itamarshamir/Projects/ear_trainer/eartrainer/eartrainer_Cpp/python


In [2]:
# import sys
import pathlib

PROJECT_ROOT = pathlib.Path().resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

In [3]:
# MIDI playback helper
# import ctypes 
# import fluidsynth; print('pyfluidsynth OK')
if 'midi_player' in globals():
    try:
        midi_player.close()
    except Exception:
        pass

from eartrainer import SimpleMidiPlayer

midi_player = SimpleMidiPlayer()
midi_player = SimpleMidiPlayer(soundfont_path="/Users/itamarshamir/Projects/ear_trainer/soundfonts/GM.sf2")

def play_prompt(bundle):
    """Play the prompt for a QuestionBundle or AssistBundle."""
    prompt = getattr(bundle, 'prompt', None)
    if prompt is None:
        print('No prompt available for playback.')
        return
    midi_player.play_prompt(prompt)

def stop_audio():
    """Stop any sustained notes without tearing down the synth."""
    midi_player.stop_all()


fluidsynth: warning: SDL3 not initialized, SDL3 audio driver won't be usable. Have you called SDL_Init(SDL_INIT_AUDIO) ?
fluidsynth: warning: SDL3 not initialized, SDL3 audio driver won't be usable. Have you called SDL_Init(SDL_INIT_AUDIO) ?


In [4]:
from eartrainer.session_engine import SessionEngine
from eartrainer import models

print("SessionEngine ready")

seed = 1234
engine = SessionEngine()
spec = models.SessionSpec(
        version="v1",
        key="F major",
        drill_kind="chord",
        mode="level_inspector",
        seed=seed,
    )
session_id = engine.create_session(spec)
s = engine.level_catalog_overview(session_id)
print(s)
engine.level_catalog_levels(session_id)

SessionEngine ready
builtin levels
  Level 1: tier 0 -> [PATHWAYS_1_4] | tier 1 -> [PATHWAYS_5_7]
  Level 2: tier 0 -> [NOTE_1_4] | tier 1 -> [NOTE_5_7]
  Level 3: tier 0 -> [NOTE_TO_TONIC] | tier 1 -> [NOTE_AFTER_TONIC]
  Level 11: tier 0 -> [PATHWAYS_2OCT] | tier 1 -> [NOTE_WITH_TONIC_2OCT] | tier 2 -> [NOTE_2OCT]
  Level 111: tier 0 -> [MELODY60_STEP1] | tier 1 -> [MELODY60_STEP2] | tier 2 -> [MELODY60]
  Level 112: tier 0 -> [MELODY90_STEP1] | tier 1 -> [MELODY90_STEP2] | tier 2 -> [MELODY90]
  Level 113: tier 0 -> [MELODY120_STEP1] | tier 1 -> [MELODY120_STEP2] | tier 2 -> [MELODY120]
  Level 220: tier 0 -> [CHORD_TRIADS_PIANO] | tier 1 -> [CHORD_TRIADS_STRINGS] | tier 2 -> [CHORD_TRIADS_PIANO_ROOT_HELPER]
  Level 221: tier 0 -> [CHORD_INVERSIONS_PIANO] | tier 1 -> [CHORD_INVERSIONS_STRINGS] | tier 2 -> [CHORD_INVERSIONS_STRINGS_ROOT_HELPER]
  Level 222: tier 0 -> [CHORD_EXTENDED_TRIADS] | tier 1 -> [CHORD_EXTENDED_STRINGS] | tier 2 -> [CHORD_EXTENDED_PIANO_FAST] | tier 3 -> [CHOR

'Levels: 1 (0,1), 2 (0,1), 3 (0,1), 11 (0,1,2), 111 (0,1,2), 112 (0,1,2), 113 (0,1,2), 220 (0,1,2), 221 (0,1,2), 222 (0,1,2,3), 223 (0,1,2,3,4)'

In [5]:
levels = engine.level_catalog_entries(spec)
for level in levels:
    print(level.label)

1-0: PATHWAYS_1_4
1-1: PATHWAYS_5_7
2-0: NOTE_1_4
2-1: NOTE_5_7
3-0: NOTE_TO_TONIC
3-1: NOTE_AFTER_TONIC
11-0: PATHWAYS_2OCT
11-1: NOTE_WITH_TONIC_2OCT
11-2: NOTE_2OCT
111-0: MELODY60_STEP1
111-1: MELODY60_STEP2
111-2: MELODY60
112-0: MELODY90_STEP1
112-1: MELODY90_STEP2
112-2: MELODY90
113-0: MELODY120_STEP1
113-1: MELODY120_STEP2
113-2: MELODY120
220-0: CHORD_TRIADS_PIANO
220-1: CHORD_TRIADS_STRINGS
220-2: CHORD_TRIADS_PIANO_ROOT_HELPER
221-0: CHORD_INVERSIONS_PIANO
221-1: CHORD_INVERSIONS_STRINGS
221-2: CHORD_INVERSIONS_STRINGS_ROOT_HELPER
222-0: CHORD_EXTENDED_TRIADS
222-1: CHORD_EXTENDED_STRINGS
222-2: CHORD_EXTENDED_PIANO_FAST
222-3: CHORD_EXTENDED_STRINGS_ROOT_HELPER
223-0: CHORD_STRINGS_LONG_SUSTAIN
223-1: CHORD_STRINGS_FADING
223-2: CHORD_STRINGS_SMOOTH_TOP
223-3: CHORD_PIANO_WITH_ROOT_HINT
223-4: CHORD_PIANO_SMOOTH_TOP


In [6]:
engine.set_level(session_id, 1, 1)
key = engine.session_key(session_id)
print("Session key:", key)
prompt = engine.orientation_prompt(session_id)
print("Orientation prompt:", prompt)
# midi_player.play_prompt(prompt)
prompt.tracks[0].events

Session key: F major
Orientation prompt: MidiClip(ppq=480, tempo_bpm=96, length_ticks=3974, tracks=[MidiTrack(name='orientation', channel=0, program=0, events=[MidiEvent(t=0, type='note_on', note=53, vel=90, control=None, value=None), MidiEvent(t=346, type='note_off', note=53, vel=None, control=None, value=None), MidiEvent(t=346, type='note_on', note=55, vel=90, control=None, value=None), MidiEvent(t=691, type='note_on', note=57, vel=90, control=None, value=None), MidiEvent(t=692, type='note_off', note=55, vel=None, control=None, value=None), MidiEvent(t=1037, type='note_off', note=57, vel=None, control=None, value=None), MidiEvent(t=1037, type='note_on', note=58, vel=90, control=None, value=None), MidiEvent(t=1382, type='note_on', note=60, vel=90, control=None, value=None), MidiEvent(t=1383, type='note_off', note=58, vel=None, control=None, value=None), MidiEvent(t=1728, type='note_off', note=60, vel=None, control=None, value=None), MidiEvent(t=1728, type='note_on', note=62, vel=90, c

[MidiEvent(t=0, type='note_on', note=53, vel=90, control=None, value=None),
 MidiEvent(t=346, type='note_off', note=53, vel=None, control=None, value=None),
 MidiEvent(t=346, type='note_on', note=55, vel=90, control=None, value=None),
 MidiEvent(t=691, type='note_on', note=57, vel=90, control=None, value=None),
 MidiEvent(t=692, type='note_off', note=55, vel=None, control=None, value=None),
 MidiEvent(t=1037, type='note_off', note=57, vel=None, control=None, value=None),
 MidiEvent(t=1037, type='note_on', note=58, vel=90, control=None, value=None),
 MidiEvent(t=1382, type='note_on', note=60, vel=90, control=None, value=None),
 MidiEvent(t=1383, type='note_off', note=58, vel=None, control=None, value=None),
 MidiEvent(t=1728, type='note_off', note=60, vel=None, control=None, value=None),
 MidiEvent(t=1728, type='note_on', note=62, vel=90, control=None, value=None),
 MidiEvent(t=2074, type='note_off', note=62, vel=None, control=None, value=None),
 MidiEvent(t=2074, type='note_on', note=6

In [8]:
# from pprint import pprint
import time 
QUESTIONS = 5
timeline = []
midi_player.play_prompt(prompt)
for i in range(QUESTIONS):
    next_item = engine.next_question(session_id)
    if isinstance(next_item, models.SessionSummary):
        summary = next_item
        break

    bundle = next_item
    print(bundle.correct_answer)
    print(bundle.question)
    midi_player.play_bundle(bundle)
    time.sleep(1)
    
    metrics = models.ResultMetrics(rt_ms=1200, attempts=1, question_count=1)
    report = models.ResultReport(
        question_id=bundle.question_id,
        final_answer=bundle.correct_answer,
        correct=True,
        metrics=metrics,
    )
    timeline.append(bundle.question_id)

    next_payload = engine.submit_result(session_id, report)
    if isinstance(next_payload, models.SessionSummary):
        summary = next_payload
        break

# memory = engine.end_session(session_id)
# diagnostics = engine.adaptive_diagnostics(session_id)
# pprint(memory)
# pprint(diagnostics)

MelodyAnswer(melody=[4])
{'helper': None, 'key': 'major', 'melody': [4], 'octave': None, 'tonic': 'F major', 'tonic_midi': 53, 'type': 'melody'}
MelodyAnswer(melody=[5])
{'helper': None, 'key': 'major', 'melody': [5], 'octave': None, 'tonic': 'F major', 'tonic_midi': 53, 'type': 'melody'}
MelodyAnswer(melody=[4])
{'helper': None, 'key': 'major', 'melody': [4], 'octave': None, 'tonic': 'F major', 'tonic_midi': 53, 'type': 'melody'}
MelodyAnswer(melody=[5])
{'helper': None, 'key': 'major', 'melody': [5], 'octave': None, 'tonic': 'F major', 'tonic_midi': 53, 'type': 'melody'}
MelodyAnswer(melody=[4])
{'helper': None, 'key': 'major', 'melody': [4], 'octave': None, 'tonic': 'F major', 'tonic_midi': 53, 'type': 'melody'}


In [11]:
engine.assist(session_id, bundle.question_id, 'tonic')

AssistBundle(question_id='li-007', kind='tonic', prompt_clip=MidiClip(ppq=480, tempo_bpm=96, length_ticks=864, tracks=[MidiTrack(name='tonic', channel=0, program=0, events=[MidiEvent(t=0, type='note_on', note=53, vel=90, control=None, value=None), MidiEvent(t=864, type='note_off', note=53, vel=None, control=None, value=None)])], format='midi-clip/v1'))

In [7]:
bundle.question.payload

{'bass_degree': -10,
 'bass_midi': 31,
 'bass_offset': -14,
 'bass_voicing_id': 'strings_root_low',
 'degrees': [-10, -3, 1, 4, 8, 13],
 'quality': 'major',
 'right_hand_midi': [43, 50, 55, 62, 71],
 'right_offsets': [-7, -3, 0, 4, 9],
 'root_degree': 4,
 'tonic_midi': 60,
 'training_root_midi': 67,
 'voicing_id': 'strings_open_five_low',
 'voicing_index': 1,
 'voicing_midi': [31, 43, 50, 55, 62, 71],
 'voicings_source': 'strings_ensemble'}

In [10]:
bundle.__dict__

{'question_id': 'li-007',
 'question': TypedPayload(type='chord', payload={'bass_degree': -11, 'bass_midi': 41, 'bass_offset': -14, 'bass_voicing_id': 'strings_root_low', 'degrees': [-11, -4, 0, 5, 10, 14], 'quality': 'major', 'right_hand_midi': [53, 60, 69, 77, 84], 'right_offsets': [-7, -3, 2, 7, 11], 'root_degree': 3, 'tonic_midi': 60, 'training_root_midi': 65, 'voicing_id': 'strings_open_five_high', 'voicing_index': 2, 'voicing_midi': [41, 53, 60, 69, 77, 84], 'voicings_source': 'strings_ensemble'}),
 'correct_answer': TypedPayload(type='chord_degree', payload={'root_degree': 3}),
 'prompt': Prompt(modality='midi-clip', midi_clip=MidiClip(ppq=480, tempo_bpm=60, length_ticks=6000, tracks=[MidiTrack(name='prompt', channel=0, program=51, events=[MidiEvent(t=0, type='note_on', note=41, vel=94, control=None, value=None), MidiEvent(t=0, type='note_on', note=53, vel=94, control=None, value=None), MidiEvent(t=0, type='note_on', note=60, vel=94, control=None, value=None), MidiEvent(t=0, typ

In [50]:
bundle.__dict__

{'question_id': 'li-005',
 'question': TypedPayload(type='chord', payload={'bass_degree': -10, 'bass_midi': 31, 'bass_offset': -14, 'bass_voicing_id': 'root_low', 'degrees': [-10, 1, 4, 6], 'quality': 'major', 'right_hand_midi': [50, 55, 59], 'right_offsets': [-3, 0, 2], 'root_degree': 4, 'tonic_midi': 60, 'voicing_id': 'drop2_cluster', 'voicing_index': 4, 'voicing_midi': [31, 50, 55, 59], 'voicings_source': 'builtin_diatonic_triads'}),
 'correct_answer': TypedPayload(type='chord_degree', payload={'root_degree': 4}),
 'prompt': Prompt(modality='midi-clip', midi_clip=MidiClip(ppq=480, tempo_bpm=90, length_ticks=648, tracks=[MidiTrack(name='bass', channel=1, program=0, events=[MidiEvent(t=0, type='note_on', note=31, vel=90, control=None, value=None), MidiEvent(t=648, type='note_off', note=31, vel=None, control=None, value=None)]), MidiTrack(name='right', channel=0, program=0, events=[MidiEvent(t=0, type='note_on', note=50, vel=90, control=None, value=None), MidiEvent(t=0, type='note_on',

In [9]:
bundle.prompt.midi_clip.tracks[0].events

[MidiEvent(t=0, type='note_on', note=36, vel=94, control=None, value=None),
 MidiEvent(t=0, type='note_on', note=48, vel=94, control=None, value=None),
 MidiEvent(t=0, type='note_on', note=55, vel=94, control=None, value=None),
 MidiEvent(t=0, type='note_on', note=64, vel=94, control=None, value=None),
 MidiEvent(t=0, type='note_on', note=72, vel=94, control=None, value=None),
 MidiEvent(t=0, type='note_on', note=79, vel=94, control=None, value=None),
 MidiEvent(t=1440, type='note_off', note=36, vel=None, control=None, value=None),
 MidiEvent(t=1440, type='note_off', note=48, vel=None, control=None, value=None),
 MidiEvent(t=1440, type='note_off', note=55, vel=None, control=None, value=None),
 MidiEvent(t=1440, type='note_off', note=64, vel=None, control=None, value=None),
 MidiEvent(t=1440, type='note_off', note=72, vel=None, control=None, value=None),
 MidiEvent(t=1440, type='note_off', note=79, vel=None, control=None, value=None)]

In [15]:
bundle.correct_answer.payload

{'root_degree': 4}

In [20]:
bundle.question.payload['degrees']

[-10, -3, 1, 4, 8, 13]

In [22]:
engine.debug_state(session_id)

{'drill_kind': 'chord',
 'level_catalog_overview': 'builtin levels\n  Level 1: tier 0 -> [PATHWAYS_1_4] | tier 1 -> [PATHWAYS_5_7]\n  Level 2: tier 0 -> [NOTE_1_4] | tier 1 -> [NOTE_5_7]\n  Level 3: tier 0 -> [NOTE_TO_TONIC] | tier 1 -> [NOTE_AFTER_TONIC]\n  Level 11: tier 0 -> [PATHWAYS_2OCT] | tier 1 -> [NOTE_WITH_TONIC_2OCT] | tier 2 -> [NOTE_2OCT]\n  Level 111: tier 0 -> [MELODY60_STEP1] | tier 1 -> [MELODY60_STEP2] | tier 2 -> [MELODY60]\n  Level 112: tier 0 -> [MELODY90_STEP1] | tier 1 -> [MELODY90_STEP2] | tier 2 -> [MELODY90]\n  Level 113: tier 0 -> [MELODY120_STEP1] | tier 1 -> [MELODY120_STEP2] | tier 2 -> [MELODY120]\n  Level 220: tier 0 -> [CHORD_TRIADS_PIANO] | tier 1 -> [CHORD_TRIADS_STRINGS] | tier 2 -> [CHORD_TRIADS_PIANO_ROOT_HELPER]\n  Level 221: tier 0 -> [CHORD_INVERSIONS_PIANO] | tier 1 -> [CHORD_INVERSIONS_STRINGS] | tier 2 -> [CHORD_INVERSIONS_STRINGS_ROOT_HELPER]\n  Level 222: tier 0 -> [CHORD_EXTENDED_TRIADS] | tier 1 -> [CHORD_EXTENDED_STRINGS] | tier 2 -> [CH

In [24]:
engine.orientation_prompt(session_id)

Prompt(modality='midi-clip', midi_clip=MidiClip(ppq=480, tempo_bpm=96, length_ticks=2859, tracks=[MidiTrack(name='prompt', channel=0, program=0, events=[MidiEvent(t=0, type='note_on', note=60, vel=90, control=None, value=None), MidiEvent(t=246, type='note_off', note=60, vel=None, control=None, value=None), MidiEvent(t=246, type='note_on', note=62, vel=90, control=None, value=None), MidiEvent(t=492, type='note_off', note=62, vel=None, control=None, value=None), MidiEvent(t=492, type='note_on', note=64, vel=90, control=None, value=None), MidiEvent(t=738, type='note_off', note=64, vel=None, control=None, value=None), MidiEvent(t=738, type='note_on', note=65, vel=90, control=None, value=None), MidiEvent(t=984, type='note_off', note=65, vel=None, control=None, value=None), MidiEvent(t=984, type='note_on', note=67, vel=90, control=None, value=None), MidiEvent(t=1230, type='note_off', note=67, vel=None, control=None, value=None), MidiEvent(t=1230, type='note_on', note=69, vel=90, control=None

## EXTRA DETAILS
The levels are organized by track and phase. Tracks are associated with drill kind: meldoy, degree, chord etc. Phases are packets of (at most 10) levels for each track. 

Each level has a 3 digit number. The hundred digit is the track and the tens digit is the phase. Level with the same phase are learned together. 

The user has a current level for each track. Given the levels of each track 
